In [ ]:
# This package is required for Watson Discovery 
# You might need to run the command below once if you don't have this package installed.
#!pip install --upgrade ibm-watson

In [ ]:
import json

from ibm_watson import DiscoveryV2
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

# WML python SDK
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes, DecodingMethods

## 1. Watson Discovery Configuration

In [ ]:
authenticator = IAMAuthenticator('XXXXX')
discovery = DiscoveryV2(
    version='2020-08-30',
    authenticator=authenticator
)

discovery.set_service_url('https://api.us-south.discovery.watson.cloud.ibm.com/instances/9b45b9ef-5a76-4921-91cd-8de6226d87e5')

## 2. Watson Discovery Search 

In [ ]:
question = 'I’m interested in IBM’s effect on the environment.  What efforts have they been making in sustainability??'

In [ ]:
response = discovery.query(
  project_id='XXXXX',
  collection_ids = ['XXXXX'],
  passages = {'enabled': True, 
              'max_per_document': 5,
             'find_answers': True},
  natural_language_query = question
).get_result()

with open('data.json', 'w') as f:
    json.dump(response, f)

In [ ]:
# inspecting the key fields in the WD output
response.keys()

In [ ]:
# only one relevant document (because one document in the collection)
len(response['results'])

In [ ]:
# removing some tags
passages = response['results'][0]['document_passages']
passages = [p['passage_text'].replace('<em>', '').replace('</em>', '').replace('\n','') for p in passages]
passages

In [ ]:
# concatenating passages
context = '\n '.join(passages)
context

## 3. Creating Prompt

In [ ]:
# https://huggingface.co/blog/llama2#how-to-prompt-llama-2

prompt = \
"<s>[INST] <<SYS>> \
Please answer the following question in one sentence using this text. \
If the question is unanswerable, say 'unanswerable'. \
If you responded to the question, don't say 'unanswerable'. \
Do not include information that's not relevant to the question. \
Do not answer other questions. \
Make sure the language used is English.'\
Do not use repetitions' \
Question:" + question + \
'<</SYS>>' + context + '[/INST]'

#complete_prompt = context + instruction + question

print("----------------------------------------------------------------------------------------------------")
print("*** Prompt:" + prompt + "***")
print("----------------------------------------------------------------------------------------------------")

In [ ]:
def get_model(model_type,max_tokens,min_tokens,decoding,temperature):#, repetition_penalty):

    generate_params = {
        GenParams.MAX_NEW_TOKENS: max_tokens,
        GenParams.MIN_NEW_TOKENS: min_tokens,
        GenParams.DECODING_METHOD: decoding,
        GenParams.TEMPERATURE: temperature,
    }
    
    model = Model(
        model_id=model_type,
        params=generate_params,
        credentials={
            "apikey": api_key,
            "url": url
        },
        project_id=watsonx_project_id
        )

    return model

## 4. Configuring watsonx.ai

In [ ]:
# URL of the hosted LLMs is hardcoded because at this time all LLMs share the same endpoint
url = "https://us-south.ml.cloud.ibm.com"

# Replace with your watsonx project id (look up in the project Manage tab)
watsonx_project_id = "XXXXX"
# Replace with your IBM Cloud key
api_key = "XXXXX"

In [ ]:
#model_type = "meta-llama/llama-2-70b-chat"
model_type = "meta-llama/llama-3-1-70b-instruct"
max_tokens = 100
min_tokens = 50
decoding = DecodingMethods.GREEDY
temperature = 0.7

# Get the watsonx model
model = get_model(model_type, max_tokens, min_tokens, decoding, temperature)

## 5. Answer Generation

In [ ]:
generated_response = model.generate(prompt)
response_text = generated_response['results'][0]['generated_text']

# print model response
print("--------------------------------- Generated response -----------------------------------")
print(response_text)
print("*********************************************************************************************")